In [48]:
import pandas as pd
import numpy as np
from sklearn import 

In [49]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [50]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [51]:
train.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [52]:
#precisamos de uma forma de ver todos os valores nullos ou faltantes, porém a base de dados e enorme para ver acima, por isso
train_null = train.isnull().sum() 
percent = train_null/ train.isnull().count()

In [53]:
#agora vamos plotar a quantidade de numeros nulos pelo total
grafico_nulos = pd.concat([train_null,percent], axis = 1, keys = ['Total','porcentagem_nulos'])
grafico_nulos.shape

(81, 2)

In [54]:
grafico_nulos.head(81).style.background_gradient(cmap='autumn')

,Total,porcentagem_nulos
Id,0,0.000000
MSSubClass,0,0.000000
MSZoning,0,0.000000
LotFrontage,259,0.177397
LotArea,0,0.000000
Street,0,0.000000
Alley,1369,0.937671
LotShape,0,0.000000
LandContour,0,0.000000
Utilities,0,0.000000


In [55]:
valores_faltantes = (grafico_nulos["Total"]>8)
grafico_nulos = grafico_nulos.loc[valores_faltantes]

In [32]:
train = train.drop(columns=grafico_nulos.index)

In [33]:
train = train.fillna('Unknown')
test = test.fillna('Unknown')

In [34]:
for coluna in train:
    train[coluna] = train[coluna].replace('Unknown',train[coluna].mode()[0])
for coluna in test:
    test[coluna] = test[coluna].replace('Unknown',test[coluna].mode()[0])
print(train.shape)
print(test.shape)

(1460, 65)
(1459, 80)


In [35]:
#Ver os tipos das variaveis
train.dtypes

Id                int64
MSSubClass        int64
MSZoning         object
LotArea           int64
Street           object
                  ...  
MoSold            int64
YrSold            int64
SaleType         object
SaleCondition    object
SalePrice         int64
Length: 65, dtype: object

In [36]:
def converte(df):
    df['MSSubClass'] = df['MSSubClass'].apply(str)
    df['OverallCond'] = df['OverallCond'].astype(str)
    df['MoSold'] = df['MoSold'].astype(str)
converte(train)
converte(test)

In [37]:
x_train = train.copy().drop(columns = ["Id","SalePrice"] )
y_train = train["SalePrice"].copy()
x_test  = test.copy().drop(columns = ["Id"] )

In [44]:
scaler = MinMaxScaler(feature_range = (0, 1)) # faz as escalas que precisamos para a machine learning variem de 0 ate 1

In [45]:
all_regr_models = [
    LinearRegression(),
    Ridge(), 
    RidgeCV(),
    LassoCV(max_iter=100000),
    ElasticNetCV()  
]

In [46]:
# there we will store RMSEs
all_rmse_train = {}

# there we will store accuracies
all_acc_train = {}

In [43]:
for model in all_regr_models:
    
    model_name = model.__class__.__name__ 
    print("♦ ", model_name)
    
    model.fit(x_train, y_train)
    
    y_train_pred = model.predict(x_train)
    mse_train = mean_squared_error(y_train, y_train_pred)
    rmse_train = math.sqrt(mse_train)
    print("- rmse_train =", round(rmse_train,7))
    
    model_acc_train = explained_variance_score(y_train, y_train_pred)
    print("- model_acc_train =", round(model_acc_train,7))
    
    all_rmse_train[model_name] = rmse_train
    all_acc_train[model_name] = model_acc_train
    
    y_test_pred_log1p = model.predict(x_test)
    y_test_pred = np.expm1(y_test_pred_log1p)
    
    filename = model_name + '_model.pickle'
    pickle.dump(model, open(filename, 'wb'))  
    
    if (model_name == 'LassoCV'):
        submission = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': y_test_pred})
        submission.to_csv('submission.csv', index=False)
        print('! submission is successful')
        
    print()

♦  LinearRegression


ValueError: could not convert string to float: 'RL'